# Using the MANN Package to convert and prune an existing TensorFlow model

In this notebook, we utilize the MANN package on an existing TensorFlow model to convert existing layers to MANN layers and then prune the model using RSN2.

In [ ]:
# Load the MANN package and TensorFlow
import tensorflow as tf
import mann

In [ ]:
# Load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_test = x_test/255

# Load the model to be used
efficientnet_model = tf.keras.applications.EfficientNetB0(
    include_top = False,
    weights = None,
    input_shape = x_train.shape[1:]
)

In [ ]:
model = tf.keras.models.Sequential()
model.add(efficientnet_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation = 'relu'))
model.add(tf.keras.layers.Dense(100, activation = 'relu'))
model.add(tf.keras.layers.Dense(100, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

model.summary()

In [ ]:
converted_model = mann.utils.add_layer_masks(model)
converted_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)
converted_model.summary()

In [ ]:
callback = mann.utils.ActiveSparsification(
    0.65
)
model.fit(
    x_train,
    y_train,
    epochs = 1000,
    callbacks = [callback],
    validation_split = 0.2,
    batch_size = 256
)